# TFM. Llamadas Call Center
## MVP

## Importacion de librerias 

In [1]:
import pandas as pd
pd.options.display.max_columns = None

### Importacion de los datos de llamadas por oficinas

In [2]:
df_calls = pd.read_csv('data/calls.csv', low_memory=False)

In [3]:
df_calls.columns

Index(['Unnamed: 0', 'idTransaccion', 'tInicio', 'tFinal', 'estado',
       'tProximoContacto', 'tReplanificacion', 'nTAdmin', 'nTAdminExcedido',
       'nTQ', 'idSujeto', 'sIdOriginal', 'sNombreCliente', 'sApellido1Cliente',
       'sApellido2Cliente', 'sTelefonoCliente', 'idServicio', 'idCampanya',
       'idAgente', 'idSegmento', 'sNombreSegmento', 'sAtributoSegmento',
       'sFiltroSegmento', 'sDescripcionFinal', 'sDescripcionFinal.1',
       'idFinal', 'sDescripcionFinal.2', 'sFiltroFinal', 'tActualizado',
       'tCreacion', 'nOrigenTransaccion'],
      dtype='object')

In [4]:
df_calls['sTelefonoCliente'][df_calls['sTelefonoCliente'].isnull() == True] 

0         NaN
1         NaN
1634      NaN
21123     NaN
25623     NaN
25659     NaN
61933     NaN
71745     NaN
71786     NaN
82418     NaN
82704     NaN
101157    NaN
108384    NaN
111725    NaN
129667    NaN
133307    NaN
135941    NaN
138519    NaN
142555    NaN
Name: sTelefonoCliente, dtype: object

In [5]:
df_calls = df_calls[df_calls['sTelefonoCliente'].notna()]

In [6]:
df_calls = df_calls[['tCreacion','tInicio','tFinal','sTelefonoCliente','idCampanya']]
df_calls.head()

,tCreacion,tInicio,tFinal,sTelefonoCliente,idCampanya
2,2020-03-01 13:42:29.567,NaN,2020-03-01 13:42:44.647,918298287,100000021.0
3,2020-03-01 18:48:01.543,NaN,2020-03-01 18:48:08.430,664401666,100000021.0
4,2020-03-02 02:46:55.637,NaN,2020-03-02 02:47:07.040,918298287,100000015.0
5,2020-03-02 07:28:57.743,NaN,2020-03-02 07:29:06.180,653312359,100000021.0
6,2020-03-02 08:05:45.267,2020-03-02 08:06:37.973,2020-03-02 08:08:33.807,971886546,100000021.0


In [7]:
df_calls['tCreacion'] = pd.to_datetime(df_calls['tCreacion'])
df_calls['tInicio'] = pd.to_datetime(df_calls['tInicio'])
df_calls['tFinal'] = pd.to_datetime(df_calls['tFinal'])

df_calls['Date'] = df_calls['tCreacion'].dt.date
df_calls['Date'] = df_calls['Date'].astype('str')

df_calls['dateCreacion'] = df_calls['tCreacion'].dt.date
df_calls['dateCreacion'] = df_calls['dateCreacion'].astype('str')

df_calls['dateInicio'] = df_calls['tInicio'].dt.date
df_calls['dateInicio'] = df_calls['dateInicio'].astype('str') 

In [8]:
df_calls.head()

,tCreacion,tInicio,tFinal,sTelefonoCliente,idCampanya,Date,dateCreacion,dateInicio
2,2020-03-01 13:42:29.567,NaT,2020-03-01 13:42:44.647,918298287,100000021.0,2020-03-01,2020-03-01,NaT
3,2020-03-01 18:48:01.543,NaT,2020-03-01 18:48:08.430,664401666,100000021.0,2020-03-01,2020-03-01,NaT
4,2020-03-02 02:46:55.637,NaT,2020-03-02 02:47:07.040,918298287,100000015.0,2020-03-02,2020-03-02,NaT
5,2020-03-02 07:28:57.743,NaT,2020-03-02 07:29:06.180,653312359,100000021.0,2020-03-02,2020-03-02,NaT
6,2020-03-02 08:05:45.267,2020-03-02 08:06:37.973,2020-03-02 08:08:33.807,971886546,100000021.0,2020-03-02,2020-03-02,2020-03-02


In [9]:
df_calls = df_calls[['Date','dateCreacion','sTelefonoCliente','idCampanya']]

In [10]:
df_calls.drop_duplicates(inplace=True)

### Filtramos campaña

In [11]:
#df_calls = df_calls[ df_calls['idCampanya'] == 100000015.0 ]

In [12]:
df_calls = df_calls.groupby('Date').count()
df_calls = df_calls[['dateCreacion']]
df_calls = df_calls[df_calls.dateCreacion > 20]

In [13]:
df_calls.to_pickle('./data/df_Call')